In [1]:
import pandas as pd
import os

In [2]:
tr = pd.read_csv('../../preprocess/train_trade_pre.csv')
te = pd.read_csv('../../preprocess/test_trade_pre.csv')

In [3]:
# train, test 데이터를 한꺼번에 정리하기 위해서 합해준다.
total = pd.concat([tr,te]).sort_values(by=list(tr.columns))
total.drop_duplicates(inplace=True)
total.head()

,trade_week,trade_day,trade_time,source_acc_id,target_acc_id,item_type,item_amount
0,1,1,00:00:00,3c3baeef181bd1248e182851c47a39d16c7c343fd38ddc...,f9fd4dae996c583b6173d0dd5816db49fc03b09e9a532a...,grocery,2.0000
1,1,1,00:00:00,3c3baeef181bd1248e182851c47a39d16c7c343fd38ddc...,f9fd4dae996c583b6173d0dd5816db49fc03b09e9a532a...,money,28.5122
0,1,1,00:00:00,a4785c1f445c5b9e93e84fac2ff7709e30caca844d5c01...,1bccfbc74b6a875ecfd724288fd0f5b8f47851a8049fe9...,money,220.0000
2,1,1,00:00:02,918c5389ed6f6e7becfa06e179966c12a2bfbd5b147a6d...,065a4d4a8607317210dd1283d0f5d2d64723736fae0d14...,money,200.0000
3,1,1,00:00:02,9c8c6cf69f6b2c48e7c377bb8a0327565973651bc8dbde...,0eb20d13ef580d1ccf04d264d69e0cca19e50dc7400926...,grocery,1.0000


# trade 데이터 전처리
* 전체 기간동안 id간의 거래 횟수를 구한다.
* 각 acc_id별 주당 총 거래량을 구한다.

In [4]:
sota = total.groupby(['source_acc_id', 'target_acc_id', 'item_type', 'trade_week']).agg({'item_amount': 'sum', 'target_acc_id': 'count'})
sota.columns = ['item_amount_sum', 'trade_count']
sota = sota.reset_index(drop=False).rename(columns={'source_acc_id': 'acc_id'})

taso = total.groupby(['target_acc_id', 'source_acc_id', 'item_type', 'trade_week']).agg({'item_amount': 'sum', 'source_acc_id': 'count'})
taso.columns = ['item_amount_sum', 'trade_count']
taso = taso.reset_index(drop=False).rename(columns={'target_acc_id': 'acc_id'})

columns = ['acc_id', 'trade_week', 'source_acc_id', 'target_acc_id', 'trade_count', 'item_type', 'item_amount_sum']
to_trade = pd.merge(sota, taso, how='outer')[columns]
to_trade.head()

,acc_id,trade_week,source_acc_id,target_acc_id,trade_count,item_type,item_amount_sum
0,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,7,NaN,2d88f6a0701e8e76806a28ad6a3a0a11ba8f142323e67d...,2,grocery,521.0000
1,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,8,NaN,2d88f6a0701e8e76806a28ad6a3a0a11ba8f142323e67d...,3,grocery,718.0000
2,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,7,NaN,2d88f6a0701e8e76806a28ad6a3a0a11ba8f142323e67d...,2,money,200.4570
3,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,8,NaN,2d88f6a0701e8e76806a28ad6a3a0a11ba8f142323e67d...,3,money,300.0105
4,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,8,NaN,9a24464e55b78ab4a37d6704419d142cb191723c723a69...,1,grocery,278.0000


In [5]:
to_trade['item_type'].unique()

array(['grocery', 'money', 'gem', 'accessory', 'weapon', 'costume'],
      dtype=object)

In [6]:
grocery = to_trade[to_trade['item_type']=='grocery'].rename(columns={
    'trade_count': 'grocery_trade_count', 'item_amount_sum': 'grocery_sum'}).drop('item_type', axis=1)
money = to_trade[to_trade['item_type']=='money'].rename(columns={
    'trade_count': 'money_trade_count', 'item_amount_sum': 'money_sum'}).drop('item_type', axis=1)
gem = to_trade[to_trade['item_type']=='gem'].rename(columns={
    'trade_count': 'gem_trade_count', 'item_amount_sum': 'gem_sum'}).drop('item_type', axis=1)
accessory = to_trade[to_trade['item_type']=='accessory'].rename(columns={
    'trade_count': 'accessory_trade_count', 'item_amount_sum': 'accessory_sum'}).drop('item_type', axis=1)
weapon = to_trade[to_trade['item_type']=='weapon'].rename(columns={
    'trade_count': 'weapon_trade_count', 'item_amount_sum': 'weapon_sum'}).drop('item_type', axis=1)
costume = to_trade[to_trade['item_type']=='costume'].rename(columns={
    'trade_count': 'costume_trade_count', 'item_amount_sum': 'costume_sum'}).drop('item_type', axis=1)

### acc_id 기준으로 주고 받은 내역을 만들어준다.

In [7]:
total_trade = money.merge(grocery, how='outer').merge(gem, how='outer').merge(accessory, how='outer').merge(weapon, how='outer').merge(costume, how='outer')
total_trade = total_trade.sort_values(by=list(total_trade.columns)).reset_index(drop=True)
total_trade['trade_count'] = total_trade[[i for i in list(total_trade.columns) if 'count' in i]].sum(axis=1)
total_trade.head()

,acc_id,trade_week,source_acc_id,target_acc_id,money_trade_count,money_sum,grocery_trade_count,grocery_sum,gem_trade_count,gem_sum,accessory_trade_count,accessory_sum,weapon_trade_count,weapon_sum,costume_trade_count,costume_sum,trade_count
0,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,7,2d88f6a0701e8e76806a28ad6a3a0a11ba8f142323e67d...,NaN,NaN,NaN,2.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,7,NaN,2d88f6a0701e8e76806a28ad6a3a0a11ba8f142323e67d...,2.0,200.4570,2.0,521.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,8,2d88f6a0701e8e76806a28ad6a3a0a11ba8f142323e67d...,NaN,NaN,NaN,3.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
3,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,8,9a24464e55b78ab4a37d6704419d142cb191723c723a69...,NaN,NaN,NaN,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,8,NaN,2d88f6a0701e8e76806a28ad6a3a0a11ba8f142323e67d...,3.0,300.0105,3.0,718.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0


### 전체 기간 acc_id간의 총 거래 횟수를 구한다.

In [8]:
source_count = total_trade.groupby(['acc_id', 'source_acc_id']).agg({'trade_count': 'sum'}).reset_index()
source_count.rename(columns={'source_acc_id': 'trade_acc_id', 'trade_count': 'rec_count'}, inplace=True)

target_count = total_trade.groupby(['acc_id', 'target_acc_id']).agg({'trade_count': 'sum'}).reset_index()
target_count.rename(columns={'target_acc_id': 'trade_acc_id', 'trade_count': 'give_count'}, inplace=True)

total_count = pd.merge(source_count, target_count,
                       how='outer').sort_values(by=['acc_id', 'trade_acc_id']).reset_index(drop=True).fillna(0)
total_count.head(10)

,acc_id,trade_acc_id,rec_count,give_count
0,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,2d88f6a0701e8e76806a28ad6a3a0a11ba8f142323e67d...,5.0,10.0
1,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,9a24464e55b78ab4a37d6704419d142cb191723c723a69...,1.0,2.0
2,0000548b3eecb5e8c6736f9408ddd58f49d0a553b6f1f8...,2d1a9afbecfcfe53fd715664ec479c75b95f62f1cacec0...,0.0,4.0
3,00006916672106f94f68d18e4a0532b53a40f4549d3a62...,2d88f6a0701e8e76806a28ad6a3a0a11ba8f142323e67d...,1.0,2.0
4,00006916672106f94f68d18e4a0532b53a40f4549d3a62...,5b6c6f19feabdb312ea3b990363948fff031fd6d306f49...,2.0,4.0
5,0000d57170ee9d25dfa6cc8c7fbe88d2ffb561637020a4...,9884c507d1f33aa553a316753aa78f7fec7cbafdcd0344...,1.0,2.0
6,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,3e786c089cfe9292dbdc048311552c5520773ea740ed8c...,0.0,1.0
7,00027152625b3c992d2e423b2d811d34c26a23eb3705d8...,5c3e1fa05adb6a2d5ac9b7036c7eb34c06032baddf8162...,0.0,8.0
8,000291b3c6e9c6012afed9dd99bbc834082622667fe3b7...,c8d53718b0df2bf7f2283dc1b0cb797ce3684eb099432e...,1.0,0.0
9,0002cb165b57f8ab2bea2e387a3bebcd1ee57b44229150...,06cb8f8365a8e81054dab274b524e8693d57d26f95f408...,1.0,1.0


### 각 acc_id의 주고 받은 거래량의 주당 합, 주고 받은 횟수를 구한다.

In [9]:
columns = ['money_count', 'money_sum',
           'grocery_count', 'grocery_sum',
           'gem_count', 'gem_sum',
           'accessory_count', 'accessory_sum',
           'weapon_count', 'weapon_sum',
           'costume_count', 'costume_sum']
total_trade.drop('trade_count', axis=1, inplace=True)
total_rec = total_trade.dropna(subset=['source_acc_id']).groupby(['acc_id', 'trade_week']).sum().reset_index(drop=False)
total_give = total_trade.dropna(subset=['target_acc_id']).groupby(['acc_id', 'trade_week']).sum().reset_index(drop=False)
total_rec.columns = ['acc_id', 'trade_week'] + ['rec_'+i for i in columns]
total_give.columns = ['acc_id', 'trade_week'] + ['give_'+i for i in columns]

trade_sum = pd.merge(total_rec, total_give, how='outer').sort_values(by=['acc_id', 'trade_week']).reset_index(drop=True).fillna(0)
trade_sum['rec_count'] = trade_sum.iloc[:, 2:2+len(columns):2].sum(axis=1)
trade_sum['give_count'] = trade_sum.iloc[:, 2+len(columns):2+2*len(columns):2].sum(axis=1)
trade_sum['trade_count'] = trade_sum['rec_count'] + trade_sum['give_count']

total_columns = ['acc_id', 'trade_week', 'trade_count', 'rec_count', 'give_count']
for i in range(0, len(columns), 2):
    total_columns.extend(['rec_'+columns[i], 'rec_'+columns[i+1]])
    total_columns.extend(['give_'+columns[i], 'give_'+columns[i+1]])
trade_sum = trade_sum[total_columns]

pd.set_option('display.max_columns', len(trade_sum.columns))
trade_sum.head(10)

,acc_id,trade_week,trade_count,rec_count,give_count,rec_money_count,rec_money_sum,give_money_count,give_money_sum,rec_grocery_count,rec_grocery_sum,give_grocery_count,give_grocery_sum,rec_gem_count,rec_gem_sum,give_gem_count,give_gem_sum,rec_accessory_count,rec_accessory_sum,give_accessory_count,give_accessory_sum,rec_weapon_count,rec_weapon_sum,give_weapon_count,give_weapon_sum,rec_costume_count,rec_costume_sum,give_costume_count,give_costume_sum
0,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,7,6.0,2.0,4.0,0.0,0.0,2.0,200.4570,2.0,10.0,2.0,521.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,8,12.0,4.0,8.0,0.0,0.0,4.0,400.0039,4.0,20.0,4.0,996.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0000548b3eecb5e8c6736f9408ddd58f49d0a553b6f1f8...,6,4.0,0.0,4.0,0.0,0.0,2.0,75.9711,0.0,0.0,2.0,154.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00006916672106f94f68d18e4a0532b53a40f4549d3a62...,2,9.0,3.0,6.0,0.0,0.0,3.0,306.8733,3.0,3.0,3.0,876.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0000d57170ee9d25dfa6cc8c7fbe88d2ffb561637020a4...,8,3.0,1.0,2.0,0.0,0.0,1.0,99.9681,1.0,5.0,1.0,264.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,7,1.0,0.0,1.0,0.0,0.0,1.0,30100.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,00027152625b3c992d2e423b2d811d34c26a23eb3705d8...,3,4.0,0.0,4.0,0.0,0.0,2.0,54.9217,0.0,0.0,2.0,114.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,00027152625b3c992d2e423b2d811d34c26a23eb3705d8...,4,4.0,0.0,4.0,0.0,0.0,2.0,77.8348,0.0,0.0,2.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,000291b3c6e9c6012afed9dd99bbc834082622667fe3b7...,5,1.0,1.0,0.0,0.0,0.0,0.0,0.0000,1.0,384.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0002cb165b57f8ab2bea2e387a3bebcd1ee57b44229150...,1,2.0,1.0,1.0,1.0,1780.0,1.0,500.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# trade 데이터 처리 완료

In [10]:
total_count.to_csv('trade_count.csv', index=False)
trade_sum.to_csv('trade_sum.csv', index=False)